In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torchinfo import summary
import pandas as pd
import os

In [2]:
data_path = '/Users/matthewhagg/Desktop/Life/College/AI/DiscoverAI/archive-2/Spectrogram' #looking in subfolder train

birds_dataset = datasets.ImageFolder(
    root=data_path,
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  transforms.ToTensor()
                                  ])
)
print(birds_dataset)

Dataset ImageFolder
    Number of datapoints: 32133
    Root location: /Users/matthewhagg/Desktop/Life/College/AI/DiscoverAI/archive-2/Spectrogram
    StandardTransform
Transform: Compose(
               Resize(size=(201, 81), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [3]:
class_map=birds_dataset.class_to_idx

print("\nClass category and index of the images: {}\n".format(class_map))


Class category and index of the images: {'Acrocephalus-arundinaceus': 0, 'Acrocephalus-dumetorum': 1, 'Acrocephalus-palustris': 2, 'Acrocephalus-schoenobaenus': 3, 'Aegolius-funereus': 4, 'Alauda-arvensis': 5, 'Athene-noctua': 6, 'Bubo-bubo': 7, 'Caprimulgus-europaeus': 8, 'Carduelis-carduelis': 9, 'Carpodacus-erythrinus': 10, 'Chloris-chloris': 11, 'Crex-crex': 12, 'Cuculus-canorus': 13, 'Cyanistes-caeruleus': 14, 'Emberiza-calandra': 15, 'Emberiza-cirlus': 16, 'Emberiza-citrinella': 17, 'Emberiza-hortulana': 18, 'Emberiza-schoeniclus': 19, 'Erithacus-rubecula': 20, 'Ficedula-hypoleuca': 21, 'Ficedula-parva': 22, 'Fringilla-coelebs': 23, 'Glaucidium-passerinum': 24, 'Hippolais-icterina': 25, 'Hirundo-rustica': 26, 'Linaria-cannabina': 27, 'Locustella-naevia': 28, 'Loxia-curvirostra': 29, 'Luscinia-luscinia': 30, 'Luscinia-megarhynchos': 31, 'Luscinia-svecica': 32, 'Oriolus-oriolus': 33, 'Parus-major': 34, 'Periparus-ater': 35, 'Phoenicurus-phoenicurus': 36, 'Phylloscopus-collybita': 

In [4]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(birds_dataset))
test_size = len(birds_dataset) - train_size
birds_train_dataset, birds_test_dataset = torch.utils.data.random_split(birds_dataset, [train_size, test_size])

print("Training size:", len(birds_train_dataset))
print("Testing size:",len(birds_test_dataset))

Training size: 25706
Testing size: 6427


In [5]:

from collections import Counter

# labels in training set
train_classes = [label for _, label in birds_train_dataset]
Counter(train_classes)


OSError: image file is truncated

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    birds_train_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    birds_test_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

In [ ]:
td = train_dataloader.dataset[0][0][0][0]
print(td)

tensor([0.1216, 0.1216, 0.1216, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176,
        0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176,
        0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176,
        0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176,
        0.1176, 0.1216, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1216,
        0.1216, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176,
        0.1176, 0.1176, 0.1176, 0.1176, 0.1216, 0.1176, 0.1176, 0.1176, 0.1176,
        0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1176, 0.1216, 0.1176,
        0.1176, 0.1216, 0.1176, 0.1176, 0.1216, 0.1176, 0.1176, 0.1176, 0.1176])


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [ ]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)  

model = CNNet().to(device)

In [ ]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create the training function

def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


# Create the validation/test function

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

In [ ]:
epochs = 1

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, cost, optimizer)
    test(test_dataloader, model)
print('Done!')

Epoch 1
-------------------------------


IndexError: Target 49 is out of bounds.

In [ ]:
summary(model, input_size=(15, 3, 201, 81))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Output Shape              Param #
CNNet                                    [15, 2]                   --
├─Conv2d: 1-1                            [15, 32, 197, 77]         2,432
├─Conv2d: 1-2                            [15, 64, 94, 34]          51,264
├─Dropout2d: 1-3                         [15, 64, 94, 34]          --
├─Flatten: 1-4                           [15, 51136]               --
├─Linear: 1-5                            [15, 50]                  2,556,850
├─Linear: 1-6                            [15, 2]                   102
Total params: 2,610,648
Trainable params: 2,610,648
Non-trainable params: 0
Total mult-adds (G): 3.05
Input size (MB): 2.93
Forward/backward pass size (MB): 82.80
Params size (MB): 10.44
Estimated Total Size (MB): 96.17

In [ ]:
model.eval()
test_loss, correct = 0, 0
class_map = ['no', 'yes']

with torch.no_grad():
    for batch, (X, Y) in enumerate(test_dataloader):
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        print("Predicted:\nvalue={}, class_name= {}\n".format(pred[0].argmax(0),class_map[pred[0].argmax(0)]))
        print("Actual:\nvalue={}, class_name= {}\n".format(Y[0],class_map[Y[0]]))
        break

Predicted:
value=0, class_name= no



IndexError: list index out of range